In [ ]:
import mysql.connector
print("hello")

cnx=mysql.connector.connect(host="localhost",port=3306,user="root",password="", database="lab9_adbms_iitism")
print(cnx)

def print_query(c):
    re=c.fetchall()
    for x in re:
        print(x)

cur = cnx.cursor(buffered=True)
cnx.commit()

#cur.execute("drop table running")
#cur.execute("drop table movie")
#cur.execute("drop table show_timing_seats_details")
#cur.execute("drop table theatre")
#cur.execute('''CREATE TABLE theatre (
#    id int primary key,
#    max_capacity int,
#    name varchar(255),
#    Address varchar(255),
#    City varchar(255)
#); ''')

#cur.execute("drop table movie")
print("somewhere")
cur.execute('''CREATE TABLE movie (
    id int primary key,
    name varchar(255),
    lead_actor varchar(255),
    duration int    
); ''')

cur.execute("drop table running")
cur.execute('''CREATE TABLE running (
    theatre_id int not null ,
    movie_id int not null ,
    start_date DATE not null,
    end_date DATE not null check(end_date>start_date),
    foreign key(theatre_id) references theatre(id),
    foreign key(movie_id) references movie(id),
    primary key(movie_id,theatre_id,start_date,end_date)
); ''')

#cur.execute("drop table running")

cur.execute("DROP TRIGGER IF EXISTS check_movie_dates")

cur.execute('''

    CREATE TRIGGER check_movie_dates
BEFORE INSERT ON running
FOR EACH ROW
BEGIN
    DECLARE overlap_count INT;
    
    -- Check if there are any overlapping movies
    SELECT COUNT(*)
    INTO overlap_count
    FROM running
    WHERE new.theatre_id=theatre_id AND NOT (NEW.end_date < start_date OR NEW.start_date > end_date);
    
    -- If there's an overlap, prevent insertion
    IF overlap_count > 0 THEN
        SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = 'Cannot insert. Movie dates overlap with existing movies.';
    END IF;
END;

''')

cur.execute("SELECT NOW()")
print_query(cur)


cur.execute('''
    SELECT name
    FROM running natural join movie
    WHERE CURRENT_DATE BETWEEN start_date AND end_date;
''')
print_query(cur)


#cur.execute("drop table show_timing_seats_details")
cur.execute('''CREATE TABLE show_timing_seats_details (
    theatre_id int not null ,
    show_date DATE,
    show_time_start DATETIME not null,
    show_time_end DATETIME not null,
    show_number INT not null,
    seats_booked INT not null,
    foreign key(theatre_id) references theatre(id),
    primary key(theatre_id,show_date,show_number)
); ''')

cur.execute("DROP TRIGGER IF EXISTS check_slot_overlap")

cur.execute('''

    CREATE TRIGGER check_slot_overlap
BEFORE INSERT ON show_timing_seats_details
FOR EACH ROW
BEGIN
    DECLARE overlap_count INT;
    
    -- Check if there are any overlapping movies
    SELECT COUNT(*)
    INTO overlap_count
    FROM show_timing_seats_details
    WHERE new.theatre_id=theatre_id AND NOT (NEW.show_time_end < show_time_end OR NEW.show_time_start > show_time_end);
    
    -- If there's an overlap, prevent insertion
    IF overlap_count > 0 THEN
        SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = 'Cannot insert. Movie dates overlap with existing movies.';
    END IF;
END;

''')

cur.execute('''
    INSERT INTO theatre (id, max_capacity, name, Address, City) VALUES
(1, 200, 'Royal Cinema', '12 MG Road', 'Mumbai'),
(2, 150, 'Golden Theatre', '34 Park Street', 'Kolkata'),
(3, 300, 'Silver Screens', '56 Brigade Road', 'Bangalore'),
(4, 180, 'City Plaza Multiplex', '78 Residency Road', 'Chennai'),
(5, 250, 'Palace Theatre', '90 Connaught Place', 'New Delhi');

''')


cur.execute('''
INSERT INTO movie (id, name, lead_actor, duration)
VALUES
    (1, 'Dilwale Dulhania Le Jayenge', 'Shah Rukh Khan', 181),
    (2, '3 Idiots', 'Aamir Khan', 170),
    (3, 'Baahubali: The Beginning', 'Prabhas', 159),
    (4, 'Lagaan', 'Aamir Khan', 224),
    (5, 'PK', 'Aamir Khan', 153);


''')

cur.execute('''
INSERT INTO running (theatre_id, movie_id, start_date, end_date)
VALUES 
    (1, 1, '2024-03-25', '2024-04-01'),(2, 2, '2024-03-26', '2024-04-02'),
    (3, 3, '2024-03-27', '2024-04-03'),(4, 4, '2024-03-28', '2024-04-04'),
    (5, 5, '2024-03-29', '2024-04-05');


''')


cur.execute('''
    SELECT * from running;
''')
print_query(cur)


cur.execute("DROP TRIGGER IF EXISTS capacity_filled")

cur.execute('''

    CREATE TRIGGER capacity_filled
AFTER INSERT ON show_timing_seats_details
FOR EACH ROW
BEGIN
    DECLARE overlap_count INT;
    
    -- Check if there are any overlapping movies
    SELECT COUNT(*)
    INTO overlap_count
    FROM show_timing_seats_details
    WHERE new.theatre_id=theatre_id AND NOT (NEW.show_time_end < show_time_end OR NEW.show_time_start > show_time_end);
    
    -- If there's an overlap, prevent insertion
    IF overlap_count > 0 THEN
        SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = 'Cannot insert. Movie dates overlap with existing movies.';
    END IF;
END;

''')



cur.execute('''

CREATE TABLE booking (
    booking_id INT AUTO_INCREMENT PRIMARY KEY,
    show_theatre_id INT,
    show_date DATE,
    show_number INT,
    customer_name VARCHAR(255),
    FOREIGN KEY (show_theatre_id, show_date, show_number) REFERENCES show_timing_seats_details(theatre_id, show_date, show_number)
);

''')


cur.execute('''

CREATE TRIGGER increment_seats_booked
AFTER INSERT ON booking
FOR EACH ROW
BEGIN
    DECLARE capacity INT;
    DECLARE current_seats_booked INT;

    -- Get the maximum capacity of the theatre
    SELECT max_capacity INTO capacity FROM theatre WHERE theatre_id = NEW.show_theatre_id;

    -- Get the current value of seats_booked
    SELECT seats_booked INTO current_seats_booked
    FROM show_timing_seats_details
    WHERE theatre_id = NEW.show_theatre_id
    AND show_date = NEW.show_date
    AND show_number = NEW.show_number;

    -- Increment seats_booked by 1
    SET current_seats_booked = current_seats_booked + 1;

    -- Check if seats_booked exceeds the maximum capacity
    IF current_seats_booked <= capacity THEN
        -- Update seats_booked in show_timing_seats_details
        UPDATE show_timing_seats_details
        SET seats_booked = current_seats_booked
        WHERE theatre_id = NEW.show_theatre_id
        AND show_date = NEW.show_date
        AND show_number = NEW.show_number;
    ELSE
        -- Rollback the transaction and raise an error
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Booking failed: Maximum capacity exceeded';
    END IF;
END
'''
)

cur.execute('''

CREATE TRIGGER check_dates_match
BEFORE INSERT ON show_timing_seats_details
FOR EACH ROW
BEGIN
    IF DATE(NEW.show_time_start) != NEW.show_date OR DATE(NEW.show_time_end) != NEW.show_date THEN
        SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = 'Start and end dates must match the date attribute';
    ELSEIF NEW.show_time_start <= NEW.show_time_end THEN
        SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = 'Start time must be greater than end time';
    END IF;
END;

'''
)

